## <center>Best location to open a gym in Toronto<center>

## 1. Introduction

### 1.1 Background

People in these day and age incorporate fitness into their lifestyle. thus, we can see the surge of gyms and fitness that answer to these demands. but opening a gym or other business is a tough decision, it involves making many difficult decisions such as: Who is our targeted customers, How much should be cost for the gym membership cost? Are there any competitions in the region? And one of the most important question that needs thorough answer would be what is the best location for customers to come and excercise and in turn will optimize the profitability.

### 1.2 Business Problem

Imagine a client that want to open a gym in Toronto and want our service to help find the optimum location that will benefit the business in the long run. which location in Toronto is the optimum point of interest? we first need to think about the factors that contribute to this. it would be based on income, competition and desity of people in the neighborhood can also play an important factor as well. so, to solve these problem, we will mainly use Foursquare API to get the venues location, Neighborhoods in Toronto from Wikipedia and census data from Toronto’s Open Data Portal.

### 1.3 Interest 

The targeted audiences of this project would be the business people who want to open a new gym or expand their franchised. through this study, they will have a clear overview of the locations in Toronto and can confidently target their specific clients, which will give them competitive advantage and a head start in the gym business.

### 2. Data Acquisition and Wrangling

### 2.1 Data Sources

We mainly focus on 4 data sources in this instance.
1. <a href='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'>Wikipedia</a>: We will extract the postal code, burough and neighborhoods in Toronto.
2. <a href='https://cocl.us/Geospatial_data'>Geospatial Data</a>: A geospatial data of Toronto that contains the Postal code along with latitude and longitude of neighborhoods in Toronto.
3. Foursquare API: An API call to get the locations and information of venues in toronto. (Foursquare API requires a developer account in order to log in) 
4. <a href='http://map.toronto.ca/wellbeing/#eyJ0b3Itd2lkZ2V0LWNsYXNzYnJlYWsiOsSAcGVyY2VudE9wYWNpdHnElzcwfSwiY3VzxIJtYcSTYcSXxIBuZWlnaGJvdXJob29kc8S2fcSrxIHEg8SFxIfEicSLdGFixYXEmCLEo3RpdmVUxZBJZMSXxYnEhMWPYi1pbmRpY2HEgnLFhcWIYWdzTWFwxLYiesWCbcSXMTPErHjEly04ODM3NzQ2LjDEqTc4MDnErMSnOjU0MTI5MzkuOTIyxorGmsWIxaTFpsWoxarFksSAxZjFq2lvbsSXMsSsc8WkZ2xlxYbErMWWbWVzxJtpxrbGssStxL%2FEk8SfScWlxafFqcSDTcWDxrE6IsatbsavxrHFhw%3D%3D'>Toronto Census data</a>: List of total population, household income and other info in the neighborhoods in toronto. 

### 2.2 Data Cleaning

The above data will be combined together into a single table using pandas libriry and will further apply standard scaling operation to further help with our model that will be used in this study.